In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tamil-language-corpus-for-nlp/Tamil_Articles_Corpus/Tamil_Articles_Corpus/Tamil_Articles_Corpus.txt
/kaggle/input/tamil-language-corpus-for-nlp/Tamil_News_Corpus/Tamil_news_Dataset/Tamilmurasu_dataset_06_Jan_2011_06_Jan_2020.csv
/kaggle/input/tamil-language-corpus-for-nlp/Tamil_News_Corpus/Tamil_news_Dataset/Dinamalar_dataset_2009_2019.csv
/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpus/data_lm_tawiki_8k.pkl
/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpus/model/tamil_spm_8k.vocab
/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpus/model/tamil_spm_8k.model.pth
/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpus/valid/AD_wiki_47.txt
/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpus/valid/AC_wiki_99.txt
/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpu

In [2]:
tamil_file = '../input/tamil-language-corpus-for-nlp/Tamil_Articles_Corpus/Tamil_Articles_Corpus/Tamil_Articles_Corpus.txt'
with open(tamil_file) as f: # The with keyword automatically closes the file when you are done
    print (f.read(1000))

ஒலிம்பிக் போட்டிகள் நடந்த இடங்கள் 1. 1896 - ஏதென்ஸ், கிரீஸ் 2. 1900 - பாரிஸ், பிரான்ஸ் 3. 1904 - செயின் லூயிஸ், அமெரிக்கா 4. 1908 - லண்டன்,பிரிட்டன் 5. 1912 - ஸ்டோக்ஹோம், சுவீடன் 6. 1920 - ஆண்ட்வெர்ப், பெல்ஜியம் 7. 1924 - பாரிஸ், பிரான்ஸ் 8. 1928 - ஆம்ஸ்டர்டாம், ஹாலந்து 9. 1932 - லாஸ், ஏஞ்சல்ஸ் 10. 1936 - பெர்லின், ஜெர்மனி 11. 1948 - லண்டன், இங்கிலாந்து 12. 1952 - ஹல்சின்கி, பின்லாந்து 13. 1956 - மேபோர்ன்,ஆஸ்திரேலியா 14. 1960 - ரோம், இத்தாலி 15. 1964 - டோக்கியோ, ஜப்பான் 16. 1968 - மெக்சிகோ, மெக்ஸிக்கோ 17. 1972 - மியூனிக், ஜெர்மனி 18. 1976 - மான்ட்ரியல், கனடா 19. 1980 - மாஸ்கோ, USSR 20. 1984 - லாஸ் ஏஞ்சல்ஸ், அமெரிக்கா 21. 1988 - சியோல், தென் கொரியா 22. 1992 - பார்சிலோனா, ஸ்பெயின் 23. 1996 - அட்லாண்டா, அமெரிக்கா 24. 2000 - சிட்னி, ஆஸ்திரேலியா 25. 2004 - ஏதென்ஸ், கிரீஸ் 26. 2008 - பீஜிங், சீனா 27. 2012 - லண்டன், இங்கிலாந்து 28. 2016 - ரியோ, பிரேசில் 29. 2020 - டோக்கியோ, ஜப்பான் 30. 2024 - பாரிஸ், பிரான்ஸ் 31. 2028 - லாஸ் ஏஞ்சல்ஸ், அமெரிக்கா Music ncs: https://www.youtube.com/channel/UC_aE

In [4]:
import glob

In [9]:
pwd

'/kaggle/working'

In [16]:
!cd /kaggle/working/corpus
!mkdir train
!mkdir valid

In [17]:
# Get list of training and validation text files
train_files = glob.glob("/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpus/train/*.txt")
valid_files = glob.glob("/kaggle/input/tamil-language-corpus-for-nlp/tamil-language-Corpus/tamil-language-Corpus/valid/*.txt")

def concatenate_files(file_list, output_path):
    """
    Concatenates text files into a single file with two newlines between each file's content.

    Args:
    - file_list (list of str): List of file paths to be concatenated.
    - output_path (str): Path to the output file.
    """
    try:
        with open(output_path, "wb") as outfile:
            for file_path in file_list:
                try:
                    with open(file_path, "rb") as infile:
                        outfile.write(infile.read())
                        outfile.write(b"\n\n")
                except FileNotFoundError:
                    print(f"File not found: {file_path}")
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
        print(f"Successfully concatenated files into {output_path}")
    except Exception as e:
        print(f"Error writing to {output_path}: {e}")

# Concatenate train and validation files
concatenate_files(train_files, "/kaggle/working/train/full.txt")
concatenate_files(valid_files, "/kaggle/working/valid/full_val.txt")

Successfully concatenated files into /kaggle/working/train/full.txt
Successfully concatenated files into /kaggle/working/valid/full_val.txt


In [18]:
import tokenizers
from tokenizers import Tokenizer

In [23]:
corpus_file = '/kaggle/working/train/full.txt'
tokenizer = tokenizers.SentencePieceBPETokenizer()
tokenizer.train(corpus_file)
tokenizer.save("tamil_bpe_tokenizer.json")

In [24]:
tokenizer = Tokenizer.from_file("/kaggle/working/tamil_bpe_tokenizer.json")
def compression_ratio(text, tokenizer):
    encoded = tokenizer.encode(text)
    compression_length = len(encoded.ids)
    original_length = len(text)
    return original_length/compression_length



In [25]:
sample = "சமனற்ற நீதி புத்தகத்தை வாசித்து முடித்திருக்கின்றேன். இந்த நூல் பற்றி உங்களுடன் பகிர்ந்துகொள்கின்றேன்"
encoded_text = tokenizer.encode(sample)
compression_r = compression_ratio(sample, tokenizer)
decoded_text = tokenizer.decode(encoded_text.ids)
vocab_s = len(tokenizer.get_vocab())

In [28]:
print(f"encoded: {encoded_text.tokens}")
print(f"compression ratio: {compression_r}")
print(f"decoded: {decoded_text}")
print(f"vocab size: {vocab_s}")


encoded: ['▁சம', 'ன', 'ற்ற', '▁நீதி', '▁புத்தகத்தை', '▁வாச', 'ித்து', '▁முடித்த', 'ிருக்கின்ற', 'ேன்.', '▁இந்த', '▁நூல்', '▁பற்றி', '▁உ', 'ங்களுடன்', '▁பகிர்ந்த', 'ுகொள்க', 'ின்ற', 'ேன்']
compression ratio: 5.315789473684211
decoded: சமனற்ற நீதி புத்தகத்தை வாசித்து முடித்திருக்கின்றேன். இந்த நூல் பற்றி உங்களுடன் பகிர்ந்துகொள்கின்றேன்
vocab size: 30000
